In [1]:
import folium
import folium.plugins

from matplotlib import pyplot as plt
import pandas as pd
import xml.etree.ElementTree as ET

import pandas as pd

import random

net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [2]:
parking_areas = []
parking_areas_capacity = []
root_add = ET.parse(add_xml).getroot()
for child in root_add.iter('parkingArea'):
    parking_areas.append(child.get('id'))
    parking_areas_capacity.append(child.get('roadsideCapacity'))
parking_areas_capacity = list(map(int, parking_areas_capacity)) 
len(parking_areas)

44

In [3]:
edges_all = []

root_net = ET.parse(net_xml).getroot()
for child in root_net.iter('edge'):
    if child.get('function') != 'internal':
        edges_all.append(child.get('id'))
len(edges_all)

55

In [4]:
edges_od = []
edges_od.extend([i for i in edges_all if i[3:5] == '00'])
edges_od.extend([i for i in edges_all if i[3:5] == '04'])
edges_od.extend([i for i in edges_all if i[3:5] == '23'])
edges_od.extend([i for i in edges_all if i[3:5] == '27'])
len(edges_od)

21

In [5]:
edges_o = ['01-23-NW', '25-00-NE', '26-00-NE', '27-00-NE', '01-27-SE', '02-27-SE', 
           '03-27-SE', '27-04-SW', '04-23-NW', '03-23-NW']
edges_d = [i for i in edges_od if i not in edges_o]

In [6]:
edges_o

['01-23-NW',
 '25-00-NE',
 '26-00-NE',
 '27-00-NE',
 '01-27-SE',
 '02-27-SE',
 '03-27-SE',
 '27-04-SW',
 '04-23-NW',
 '03-23-NW']

In [7]:
edges_d

['25-00-SW',
 '27-00-SW',
 '24-04-NE',
 '26-04-NE',
 '27-04-NE',
 '01-23-SE',
 '02-23-SE',
 '03-23-SE',
 '01-27-NW',
 '03-27-NW',
 '04-27-NW']

In [8]:
# edge_od_pairs = []
# from itertools import permutations 
# for i in list(permutations(edges_od, 2)): 
#     edge_od_pairs.append(i)

In [9]:
with open(rou_xml, "w") as rou:
    print("""<?xml version="1.0" encoding="UTF-8"?>
          <routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">
          """, 
          file=rou)

In [10]:
with open(rou_xml, "a") as rou:
    print('\t<vType id="passenger" vClass="passenger"></vType>', 
          file=rou)

In [11]:
with open(rou_xml, "a") as rou:
    # via="e1 e23 e7"
    for i in range(len(edges_o)):
        o = edges_o[i]
        d = [item for item in edges_d if item[:2] == o[:2] and item[:5] != o[:5]]
        if len(d) >= 1:
            d = d[0]
        else:
            d = [item for item in edges_d if item[:2] == str(int(o[:2])+1)][0]
        print('\t<flow id="f{}" begin="{}" end="{}" number="{}" from="{}" to="{}"/>'.format(i+200, 0, 2400, 
                                                                                           3, 
                                                                                           o, d), 
              file=rou)

In [12]:
parking_demand = pd.read_csv('parking_demand_dist.csv')
parking_demand

,Duration Category,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,10,0,0,0,0,1,1,3,2,1,2,2,2,3,3,2
1,20,0,0,0,1,1,3,2,4,4,4,2,3,4,2,5
2,30,0,0,1,6,8,9,9,12,10,12,10,9,7,4,9
3,60,0,0,3,10,14,10,17,18,21,15,14,14,13,9,24
4,90,0,0,1,4,5,8,7,8,7,4,4,5,5,24,0
5,120,0,0,4,9,8,8,9,11,11,5,11,14,19,24,0
6,240,2,5,12,43,34,27,29,42,25,22,22,33,32,0,0


In [13]:
# Existing occupied parking demand 

hour = 12 # simualtion hour of the day

# adding 90min to the 60min category
parking_leq_60min_num = parking_demand[str(hour-1)].values[3] + parking_demand[str(hour-1)].values[4] + parking_demand[str(hour-2)].values[5] + parking_demand[str(hour-4)].values[6]
parking_geq_60min_num = parking_demand[str(hour-1)].values[5] + parking_demand[str(hour-1)].values[6]+ parking_demand[str(hour-2)].values[6] + parking_demand[str(hour-3)].values[6]

parking_availble_dict = dict(zip(parking_areas, parking_areas_capacity))
parking_areas_availble = parking_areas.copy()

with open(rou_xml, "a") as rou:
    
    # add existing vehicles that park greater than 60 mins
    for i in range(parking_geq_60min_num):
        parking_area = random.choice(parking_areas_availble) # parking area
        parking_availble_dict[parking_area] -= 1
        if parking_availble_dict[parking_area] == 0:
            parking_areas_availble.remove(parking_area)
        d = random.choice(edges_d)
        parking_left_time = 3600
        print('''\t<trip id="e_g60_{}" depart="{}" to="{}">
		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
	</trip>'''.format(i, 0, d, parking_area, parking_left_time), file = rou)
    
    # add existing vehicles that park less than 60 mins
    for i in range(parking_leq_60min_num):
        parking_area = random.choice(parking_areas_availble) # parking area
        parking_availble_dict[parking_area] -= 1
        if parking_availble_dict[parking_area] == 0:
            parking_areas_availble.remove(parking_area)
        d = random.choice(edges_d)
        parking_left_time = (60 - random.randint(1, 60)) * 60
        print('''\t<trip id="e_l60_{}" depart="{}" to="{}">
		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
	</trip>'''.format(i, 0, d, parking_area, parking_left_time), file = rou)
    
print(parking_availble_dict)

{'01-24-NE': 7, '01-24-SW': 0, '01-25-NE': 5, '01-25-SW': 1, '01-26-SW': 5, '02-24-NE': 1, '02-24-SW': 4, '02-25-NE': 7, '02-25-SW': 4, '02-26-NE': 0, '02-26-SW': 0, '03-24-NE': 2, '03-24-SW': 2, '03-25-NE': 3, '03-25-SW': 9, '03-26-SW': 0, '04-24-NE': 8, '04-24-SW': 1, '04-25-NE': 4, '04-25-SW': 4, '04-26-NE': 5, '04-26-SW': 3, '24-01-NW': 6, '24-01-SE': 6, '24-02-NW': 5, '24-02-SE': 4, '24-03-NW': 0, '25-01-NW': 7, '25-01-SE': 6, '25-02-NW': 3, '25-02-SE': 0, '25-03-NW': 6, '26-01-NW': 0, '26-01-SE': 5, '26-02-NW': 1, '26-02-SE': 0, '26-03-NW': 0, '26-03-SE': 4, '27-01-NW': 5, '27-01-SE': 0, '27-02-NW': 3, '27-02-SE': 0, '27-03-NW': 4, '27-03-SE': 0}


In [14]:
# total capacity
sum(parking_areas_capacity)

314

In [15]:
# available parking area
sum(parking_availble_dict.values())

140

In [16]:
parking_10min_num = parking_demand[str(hour)].values[0]
parking_20min_num = parking_demand[str(hour)].values[1]
parking_30min_num = parking_demand[str(hour)].values[2]
parking_60min_num = parking_demand[str(hour)].values[3] + parking_demand[str(hour)].values[4] + parking_demand[str(hour)].values[6] 

factor = 1

parking_10min_num *= factor
parking_20min_num *= factor
parking_30min_num *= factor
parking_60min_num *= factor

parking_config_list = []

for i in range(parking_10min_num):
    pa = random.choice(parking_areas) # parking area
    o = random.choice(edges_o)
    d = random.choice(edges_d)
    dt = random.randint(1, 3600) # depart time
    pt = 10 * 60 # parking time
    parking_config_list.append((dt, o, d, pa, pt))
#         print('''\t<trip id="t_10_{}" depart="{}" from="{}" to="{}">
# 		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
# 	</trip>'''.format(i, dt, o, d, pa, pt), file = rou)

for i in range(parking_20min_num):
    pa = random.choice(parking_areas) # parking area
    o = random.choice(edges_o)
    d = random.choice(edges_d)
    dt = random.randint(1, 3600) # depart time
    pt = 20 * 60 # parking time
    parking_config_list.append((dt, o, d, pa, pt))
#         print('''\t<trip id="t_20_{}" depart="{}" from="{}" to="{}">
# 		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
# 	</trip>'''.format(i, dt, o, d, pa, pt), file = rou)

for i in range(parking_30min_num):
    pa = random.choice(parking_areas) # parking area
    o = random.choice(edges_o)
    d = random.choice(edges_d)
    dt = random.randint(1, 3600)  # depart time
    pt = 30 * 60 # parking time
    parking_config_list.append((dt, o, d, pa, pt))
#         print('''\t<trip id="t_30_{}" depart="{}" from="{}" to="{}">
# 		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
# 	</trip>'''.format(i, dt, o, d, pa, pt), file = rou)

for i in range(parking_60min_num):
    pa = random.choice(parking_areas) # parking area
    o = random.choice(edges_o)
    d = random.choice(edges_d)
    dt = random.randint(1, 3600)  # depart time
    pt = 60 * 60 # parking time
    parking_config_list.append((dt, o, d, pa, pt))
#         print('''\t<trip id="t_60_{}" depart="{}" from="{}" to="{}">
# 		 <stop index="0" parkingArea="{}" duration="{}" parking="true"/>
# 	</trip>'''.format(i, dt, o, d, pa, pt), file = rou)


        
    

In [17]:
parking_config_list.sort(key=lambda tup: tup[0])  # sorts in place
with open(rou_xml, "a") as rou:
    for i in range(len(parking_config_list)):
        dt, o, d, pa, pt = parking_config_list[i]
        print('''\t<trip id="t{}" depart="{}" from="{}" to="{}">
		<stop index="0" parkingArea="{}" duration="{}" parking="true"/>
	</trip>'''.format(i, dt, o, d, pa, pt), file = rou)

In [18]:
with open(rou_xml, "a") as rou:
    print('\n</routes>', file = rou)